<a href="https://colab.research.google.com/github/ShinAsakawa/bit/blob/main/notebooks/2022_0617bit_faster_rcnn_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
- date: 2022_0604 original file written at 2022_0319
- filename: 2022_0604bit_faster-rcnn_fine_tuning.ipynb
- ref source: https://www.kaggle.com/yerramvarun/fine-tuning-faster-rcnn-using-pytorch/notebook
---

**注**: workers=0 で動作するので時間がかかる

In [2]:
# Google Drive からデータを入手
# このセルを実行するとブラウザの別タブで Google アカウントへの認証が求められる
# Google アカウントを選択するとクリデンシャルキーが表示されるので，そのキーを
# コピーして，このセルの出力欄にある空欄に貼り付けてエンターキー (リターンキー) を押下する
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# 以下実際のデータの情報
#https://drive.google.com/file/d/1xKXbovkEQwdJefzCuaS_a351LUIuRz-1/view?usp=sharing  for  cis.twcu.ac.jp/GitHub_shared/ccap_data.tgz on Gdrive
#https://drive.google.com/file/d/1PVwPXiBiRX4Aueqc3MHuosG5mWNnrvVa/view?usp=sharing  for 2022_0605fine_tuned_bit_line_bisection_pasiphae.cpt on Gdrive

file_id = '1xKXbovkEQwdJefzCuaS_a351LUIuRz-1'
file_id = '1PVwPXiBiRX4Aueqc3MHuosG5mWNnrvVa'
file_id = '1W5jPu6xggW1y9oaNuF_SbKEm7ZnwmtQL'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('2022_0605fine_tuned_bit_line_bisection_pasiphae.cpt')

# faster-rcnn 転移学習または微調整を用いた BIT 線分 2 等分課題

[BIT] 図版を [Faster RCNN](https://arxiv.org/abs/1506.01497) で微調整して訓練

* Faster RCNNについては [Faster-RCNNの仕組みをより深く理解するために](https://medium.com/@whatdhack/a-deeper-look-how-faster-rcnn-works-84081284e1cd) の Media 参照。
* [Pytorch 公式チュートリアル文書](https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html) も参照

転移学習 transfer learning と微調整 fine tuning については，種々考え方がある。
だがここでは，[PyTorch のチュートリアル](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html) に従って，
最終層だけ入れ替えて，最終直下層以下の結合係数を固定して考える場合を転移学習と呼ぶことにする。
全層を再学習することを，微調整と呼ぶことにする。
このチュートリアルが参照にしているのは，Karpathy の スタンフォードでの授業 [cs231n の転移学習のノート](https://cs231n.github.io/transfer-learning/) である。


# 1. インストールとインポート
<!-- ## Installs and Imports -->

## 1.1 下準備

必要なライブラリのインストールなど

In [5]:
# 2022_0605 現在，下記の再インストールが必要のようだ
# また，訓練時に halt する
!pip install --upgrade albumentations
#!pip uninstall opencv-python-headless==4.5.5.62
#!pip install opencv-python-headless==4.5.2.52
!pip install opencv-python-headless --upgrade

In [6]:
import os
import sys
import shutil
import typing
import cv2
import glob
from tqdm.notebook import tqdm

import IPython
isColab = 'google.colab' in str(IPython.get_ipython())

if isColab:
    from PIL import ImageFont
    from glob import glob

    !pip install pycocotools --quiet
    !git clone https://github.com/pytorch/vision.git
    !git checkout v0.3.0

    # Download TorchVision repo to use some files from references/detection
    # os.symlink(src,dst) にした方が良いかも
    !cp vision/references/detection/utils.py ./
    !cp vision/references/detection/transforms.py ./
    !cp vision/references/detection/coco_eval.py ./
    !cp vision/references/detection/engine.py ./
    !cp vision/references/detection/coco_utils.py ./

    !pip install japanize_matplotlib
    #!pip install albumentataions  # 2022_0604 一時的に中断 colab でエラー発生のため

    # 自作ライブラリ
    !git clone https://github.com/ShinAsakawa/bit.git

    # Noto fonts のダウンロードとインストール
    !mkdir Noto_JP_fonts
    !wget https://noto-website-2.storage.googleapis.com/pkgs/NotoSerifJP.zip
    !wget https://noto-website-2.storage.googleapis.com/pkgs/NotoSansJP.zip
    !unzip NotoSerifJP.zip -d Noto_JP_fonts
    !unzip -o NotoSansJP.zip -d Noto_JP_fonts  # `-o` means overwrite
    !mv Noto_JP_fonts bit
    !mkdir data

    noto_font_dir = './bit/Noto_JP_fonts'
    notofonts_fnames = glob(os.path.join(noto_font_dir,'*otf'))
    notofonts = {fname.split('/')[-1].split('.')[0]:{'fname':fname} for fname in notofonts_fnames}
    for fontname in notofonts.keys():
        notofonts[fontname]['data'] = ImageFont.truetype(notofonts[fontname]['fname'])
else:
    # 自分のリポジトリからシンボリックリンクで代用
    for file in ['engine.py', 'utils.py', 'coco_utils.py', 'transforms.py', 'coco_eval.py']:
        if not os.path.exists(file):
            _file = os.path.join('../2020pytorch_vision.git/reference/detection/', file)
            !ln -s ../2020pytorch_vision.git/references/detection/engine.py .
            !ln -s ../2020pytorch_vision.git/references/detection/utils.py .
            !ln -s ../2020pytorch_vision.git/references/detection/coco_utils.py .
            !ln -s ../2020pytorch_vision.git/references/detection/transforms.py .
            !ln -s ../2020pytorch_vision.git/references/detection/coco_eval.py .


# DETR のサンプルプログラムを借用
COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]
COLORS = COLORS * 100

Cloning into 'vision'...
remote: Enumerating objects: 636635, done.
remote: Counting objects: 100% (1885/1885), done.
remote: Compressing objects: 100% (993/993), done.
remote: Total 636635 (delta 1612), reused 960 (delta 884), pack-reused 634750 (from 4)
Receiving objects: 100% (636635/636635), 1.18 GiB | 29.83 MiB/s, done.
Resolving deltas: 100% (595437/595437), done.
fatal: not a git repository (or any of the parent directories): .git
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 39.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for japanize_matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120257 sha256=1ec0fc807e79dd1b0ba09c2b5f6069f14038a5cc56a842b1c5312bef247f4bb4
  Stored in directory: /root/.cache/pip/wheels/da/a1/71/b8faeb93276fed10edffcca20746f1ef6f8d9e071eee8425fc
Successfully built japanize_matplotlib
Cloning into 'bit'...
remote: Enumerating objects: 337, done.
remote: Counting objects: 100% (15/15), done.
remote

## 1.2 ライブラリのインポート

In [7]:
# ライブラリのインポート
# python と機械学習のための基本ライブラリ
import os
import random
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import japanize_matplotlib

# torchvision ライブラリ
import torch
import torchvision
from torchvision import transforms as torchtrans
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import fasterrcnn_resnet50_fpn

# ヘルパライブラリをインポート
from engine import train_one_epoch, evaluate
import utils
import transforms as T

In [10]:
if isColab:
    !pip install --upgrade ipynbname

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.8 MB/s eta 0:00:00


In [11]:
%load_ext autoreload
%autoreload 2
from bit import BIT

if isColab:
    bit = BIT(fontdata=notofonts)
else:
    bit = BIT()
images, bboxes = bit.make_line_bisection_task_images(N=1, n_lines=3)

import PIL
def plot_pilimg_and_bbox(pil_img:PIL.Image.Image,
                         bboxes:list,
                         verbose:bool=False
                        ):
    """bounding box (物体を囲む四角形の境界領域のことを境界領域箱と呼ぶ): bbox
    PIL 画像を境界領域と共に表示する関数"""
    plt.figure(figsize=(10,8))
    plt.imshow(pil_img)
    ax = plt.gca()
    for (xmin, ymin, xmax, ymax), c in zip(bboxes, COLORS):
        if verbose:
            print(f'xmin:{xmin}, ymin:{ymin}')
        ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                   fill=False, color=c, linewidth=2))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
日付: 2025-05-22
HOSTNAME: 857e780d076c
ユーザ名: root
HOME: /root
ファイル名: /fileId=https%3A%2F%2Fgithub.com%2FShinAsakawa%2Fbit%2Fblob%2Fmain%2Fnotebooks%2F2022_0617bit_faster_rcnn_fine_tuning.ipynb
torch.__version__: 2.6.0+cu124


In [12]:
from bit import BIT_LineBisection
train_dataset = BIT_LineBisection(_symbols=bit.symbols, dirname='bit/data/line_bisection/train')
test_dataset = BIT_LineBisection(_symbols=bit.symbols, dirname='bit/data/line_bisection/test')

In [13]:
from bit import get_object_detection_model

num_classes = len(bit.symbols)
print(f'num_classes:{num_classes}, bit.symbols:{bit.symbols}')
model = get_object_detection_model(num_classes)

num_classes:49, bit.symbols:['<background>', '<line>', '★', 'あ', 'い', 'う', 'え', 'お', 'か', 'き', 'く', 'け', 'こ', 'さ', 'し', 'す', 'せ', 'そ', 'た', 'ち', 'つ', 'て', 'と', 'な', 'に', 'ぬ', 'ね', 'の', 'は', 'ひ', 'ふ', 'へ', 'ほ', 'ま', 'み', 'む', 'め', 'も', 'や', 'ゆ', 'よ', 'ら', 'り', 'る', 'れ', 'ろ', 'わ', 'を', 'ん']


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:01<00:00, 130MB/s]


変換前 model.roi_heads:RoIHeads(
  (box_roi_pool): MultiScaleRoIAlign(featmap_names=['0', '1', '2', '3'], output_size=(7, 7), sampling_ratio=2)
  (box_head): TwoMLPHead(
    (fc6): Linear(in_features=12544, out_features=1024, bias=True)
    (fc7): Linear(in_features=1024, out_features=1024, bias=True)
  )
  (box_predictor): FastRCNNPredictor(
    (cls_score): Linear(in_features=1024, out_features=91, bias=True)
    (bbox_pred): Linear(in_features=1024, out_features=364, bias=True)
  )
)
変換後 model.roi_heads:RoIHeads(
  (box_roi_pool): MultiScaleRoIAlign(featmap_names=['0', '1', '2', '3'], output_size=(7, 7), sampling_ratio=2)
  (box_head): TwoMLPHead(
    (fc6): Linear(in_features=12544, out_features=1024, bias=True)
    (fc7): Linear(in_features=1024, out_features=1024, bias=True)
  )
  (box_predictor): FastRCNNPredictor(
    (cls_score): Linear(in_features=1024, out_features=49, bias=True)
    (bbox_pred): Linear(in_features=1024, out_features=196, bias=True)
  )
)


In [14]:
XXX = torch.load('2022_0605fine_tuned_bit_line_bisection_pasiphae.cpt')
model.load_state_dict(XXX['model'])
#help(model)

<All keys matched successfully>

In [15]:
def torch_to_pil(img):
    """torchtensor を PIL 画像に変換する関数
    function to convert a torchtensor back to PIL image"""
    return torchtrans.ToPILImage()(img).convert('RGB')

# 可能なら GPU 上で学習させる
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
from bit import plot_img_bbox

_dataset = test_dataset
for N in range(_dataset.__len__()):
    #N = np.random.choice(len(_dataset)) # テストセットから画像を 1 枚選ぶ
    img, target = _dataset[N]
    model.eval()
    with torch.no_grad():
        prediction = model([img.to(device)])[0]

    plot_img_bbox(torch_to_pil(img), target, title="グランドトルース")
    plot_img_bbox(torch_to_pil(img), prediction, title="モデル予測")